In [36]:
import requests
import pandas as pd
import time
import os  # Import the os module to interact with the operating system
from dotenv import load_dotenv  # Import load_dotenv to load environment variables from a .env file
load_dotenv()
# Retrieve the secure values stored in the .env file
CLIENT_ID = os.getenv("CLIENT_ID")  # Get the CLIENT_ID from environment variables
CLIENT_SECRET = os.getenv("CLIENT_SECRET")  # Get the CLIENT_SECRET from environment variables



"""Petfinder API to pull listings for adoptable pets with pagination and retry logic."""

TOKEN_URL = "https://api.petfinder.com/v2/oauth2/token"


# 🔐 Step 1: Authenticate and obtain API access token
auth_data = {
    "grant_type": "client_credentials",
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET
}
response = requests.post(TOKEN_URL, data=auth_data)
access_token = response.json().get("access_token")

if not access_token:
    print(f"❌ Failed to obtain access token: {response.json()}")
    exit()

# 🏗 Step 2: Define headers for authentication
headers = {"Authorization": f"Bearer {access_token}"}

# 🐶 Step 3: Fetch adoptable pets using pagination with retry logic
PET_URL = "https://api.petfinder.com/v2/animals"
all_pets = []  # Store all pets data

page = 1  # Start from first page
while True:
    params = {"type": "dog", "location": "75001", "limit": 100, "page": page}
    success = False
    attempts = 0
    while not success and attempts < 3:
        response = requests.get(PET_URL, params=params, headers=headers)
        if response.status_code == 200:
            pet_data = response.json()["animals"]
            success = True  # Successful retrieval
        else:
            print(f"❌ Failed to fetch pets on page {page}: {response.status_code} - attempt {attempts+1}")
            attempts += 1
            time.sleep(5)  # Wait 5 seconds before retrying

    if not success:
        print(f"❌ Giving up on page {page}.")
        break

    if not pet_data:  # If no more pets are returned, stop pagination
        break

    all_pets.extend(pet_data)  # Append current batch of pets
    page += 1  # Move to next page

# 📊 Convert all pet data to a DataFrame
df_pets = pd.json_normalize(all_pets)

# 🏠 Step 4: Fetch shelter details dynamically
shelter_ids = df_pets["organization_id"].dropna().unique()
df_shelters = pd.DataFrame()

for shelter_id in shelter_ids:
    ORG_URL = f"https://api.petfinder.com/v2/organizations/{shelter_id}"
    response = requests.get(ORG_URL, headers=headers)
    if response.status_code == 200:
        shelter_data = response.json()["organization"]
        df_temp = pd.json_normalize(shelter_data)
        df_shelters = pd.concat([df_shelters, df_temp], ignore_index=True)

# 🔗 Merge Pet & Shelter DataFrames

df_pets = df_pets.rename(columns={"organization_id": "Shelter ID"})
df_shelters = df_shelters.rename(columns={"id": "Shelter ID"})
df_combined = df_pets.merge(df_shelters, on="Shelter ID", how="left")

# 💾 Step 5: Save DataFrames to Excel and CSV
df_pets.to_csv("adoptable_pets.csv", index=False)
df_shelters.to_csv("shelter_details.csv", index=False)
df_combined.to_csv("adoptable_pets_shelters.csv", index=False)
df_combined.to_excel("adoptable_pets_shelters.xlsx", index=False, sheet_name="Adoption Data")



print("✅ Data successfully saved! 🎉")
print("📂 CSV files: 'adoptable_pets.csv', 'shelter_details.csv', 'adoptable_pets_shelters.csv'")
print("📂 Excel file: 'adoptable_pets_shelters.xlsx'")
print(df_combined.head())

✅ Data successfully saved! 🎉
📂 CSV files: 'adoptable_pets.csv', 'shelter_details.csv', 'adoptable_pets_shelters.csv'
📂 Excel file: 'adoptable_pets_shelters.xlsx'
         id Shelter ID                                              url_x  \
0  76486424       TX26  https://www.petfinder.com/dog/58535015-7648642...   
1  76484744     TX2797  https://www.petfinder.com/dog/roseanne-7648474...   
2  76484066     TX2780  https://www.petfinder.com/dog/snow-76484066/tx...   
3  76484420     TX2625  https://www.petfinder.com/dog/fable-76484420/t...   
4  76484409     TX1382  https://www.petfinder.com/dog/dory-76484409/tx...   

  type species    age  gender    size   coat  \
0  Dog     Dog  Adult    Male   Small   None   
1  Dog     Dog  Young  Female  Medium   None   
2  Dog     Dog   Baby    Male  Medium   None   
3  Dog     Dog  Adult  Female   Large  Short   
4  Dog     Dog   Baby  Female  Medium   None   

                                                tags  ... hours.sunday  \
0           

In [ ]:
df_pets
df_shelters
df_combined

In [56]:
df_pets.dtypes

id                                int64
Shelter ID                       object
url                              object
type                             object
species                          object
age                              object
gender                           object
size                             object
coat                             object
tags                             object
name                             object
description                      object
organization_animal_id           object
photos                           object
primary_photo_cropped           float64
videos                           object
status                           object
status_changed_at                object
published_at                     object
distance                        float64
breeds.primary                   object
breeds.secondary                 object
breeds.mixed                       bool
breeds.unknown                     bool
colors.primary                   object


In [57]:
df_shelters.dtypes

Shelter ID                object
name                      object
email                     object
phone                     object
url                       object
website                   object
mission_statement         object
photos                    object
distance                  object
address.address1          object
address.address2          object
address.city              object
address.state             object
address.postcode          object
address.country           object
hours.monday              object
hours.tuesday             object
hours.wednesday           object
hours.thursday            object
hours.friday              object
hours.saturday            object
hours.sunday              object
adoption.policy           object
adoption.url              object
social_media.facebook     object
social_media.twitter      object
social_media.youtube      object
social_media.instagram    object
social_media.pinterest    object
_links.self.href          object
_links.ani

In [82]:
df_pets=pd.read_csv('adoptable_pets.csv')
df_shelters=pd.read_csv('shelter_details.csv')


# 🔎 Essential columns to keep for pet adoption analysis
essential_pet_columns = [
    "id","Shelter ID", "type", "species", "age", "gender", "size", "coat",
    "breeds.primary", "breeds.secondary", "breeds.mixed", "breeds.unknown",
    "colors.primary", "colors.secondary", "colors.tertiary",
    "attributes.spayed_neutered", "attributes.house_trained", "attributes.special_needs",
    "attributes.shots_current", "environment.children", "environment.dogs", "environment.cats",
    "contact.address.city", "contact.address.state", "contact.address.postcode",
    "status", "status_changed_at", "published_at", "distance",
    "primary_photo_cropped.full", "description"
]

# 🏡 Essential columns for shelter analysis
essential_shelter_columns = [
    "Shelter ID", "name", "email", "phone", "url", "website", 
    "mission_statement", "distance", "address.city", "address.state", "address.postcode",
    "hours.monday", "hours.tuesday", "hours.wednesday", "hours.thursday", "hours.friday",
    "hours.saturday", "hours.sunday", "adoption.policy", "adoption.url",
    "social_media.facebook", "social_media.twitter", "social_media.instagram"
]

# 🎯 Remove unnecessary columns
df_pets = df_pets[essential_pet_columns]
df_shelters = df_shelters[essential_shelter_columns]

# ✅ Check new structure
print("Updated df_pets columns:", df_pets.columns)
print("Updated df_shelters columns:", df_shelters.columns)

# 🔗 Merge Pet & Shelter DataFrames using "Shelter ID"
df_combined_essential_clm = df_pets.merge(df_shelters, on="Shelter ID", how="left")

# ✅ Check the new DataFrame structure
print("Merged DataFrame Columns:", df_combined_essential_clm .columns)

# 🏗 Save the combined dataset for analysis
df_combined_essential_clm.to_csv("adoptable_pets_shelters_final.csv", index=False)
df_combined_essential_clm.to_excel("adoptable_pets_shelters_final.xlsx", index=False, sheet_name="Adoption Data")

print("✅ Combined dataset saved successfully! 🎉")


Updated df_pets columns: Index(['id', 'Shelter ID', 'type', 'species', 'age', 'gender', 'size', 'coat',
       'breeds.primary', 'breeds.secondary', 'breeds.mixed', 'breeds.unknown',
       'colors.primary', 'colors.secondary', 'colors.tertiary',
       'attributes.spayed_neutered', 'attributes.house_trained',
       'attributes.special_needs', 'attributes.shots_current',
       'environment.children', 'environment.dogs', 'environment.cats',
       'contact.address.city', 'contact.address.state',
       'contact.address.postcode', 'status', 'status_changed_at',
       'published_at', 'distance', 'primary_photo_cropped.full',
       'description'],
      dtype='object')
Updated df_shelters columns: Index(['Shelter ID', 'name', 'email', 'phone', 'url', 'website',
       'mission_statement', 'distance', 'address.city', 'address.state',
       'address.postcode', 'hours.monday', 'hours.tuesday', 'hours.wednesday',
       'hours.thursday', 'hours.friday', 'hours.saturday', 'hours.sunday',
 

In [83]:
df_copy=df_combined_essential_clm
print(df_copy.columns)
df_copy.nunique()

Index(['id', 'Shelter ID', 'type', 'species', 'age', 'gender', 'size', 'coat',
       'breeds.primary', 'breeds.secondary', 'breeds.mixed', 'breeds.unknown',
       'colors.primary', 'colors.secondary', 'colors.tertiary',
       'attributes.spayed_neutered', 'attributes.house_trained',
       'attributes.special_needs', 'attributes.shots_current',
       'environment.children', 'environment.dogs', 'environment.cats',
       'contact.address.city', 'contact.address.state',
       'contact.address.postcode', 'status', 'status_changed_at',
       'published_at', 'distance_x', 'primary_photo_cropped.full',
       'description', 'name', 'email', 'phone', 'url', 'website',
       'mission_statement', 'distance_y', 'address.city', 'address.state',
       'address.postcode', 'hours.monday', 'hours.tuesday', 'hours.wednesday',
       'hours.thursday', 'hours.friday', 'hours.saturday', 'hours.sunday',
       'adoption.policy', 'adoption.url', 'social_media.facebook',
       'social_media.twitter

id                            7975
Shelter ID                     326
type                             1
species                          1
age                              4
gender                           2
size                             4
coat                             6
breeds.primary                 170
breeds.secondary               135
breeds.mixed                     2
breeds.unknown                   1
colors.primary                  15
colors.secondary                15
colors.tertiary                 15
attributes.spayed_neutered       2
attributes.house_trained         2
attributes.special_needs         2
attributes.shots_current         2
environment.children             2
environment.dogs                 2
environment.cats                 2
contact.address.city           145
contact.address.state            2
contact.address.postcode       221
status                           1
status_changed_at             6105
published_at                  5683
distance_x          

In [84]:
print(df_copy.isnull().sum())

id                               0
Shelter ID                       0
type                             0
species                          0
age                              0
gender                           0
size                             0
coat                          5931
breeds.primary                   0
breeds.secondary              4630
breeds.mixed                     0
breeds.unknown                   0
colors.primary                4799
colors.secondary              6660
colors.tertiary               7912
attributes.spayed_neutered       0
attributes.house_trained         0
attributes.special_needs         0
attributes.shots_current         0
environment.children          5139
environment.dogs              4362
environment.cats              6415
contact.address.city             0
contact.address.state            0
contact.address.postcode         0
status                           0
status_changed_at                0
published_at                     0
distance_x          

In [85]:
df_copy = df_copy.drop(columns=["coat", "breeds.secondary", "colors.primary", "colors.secondary", 
                                        "colors.tertiary", "environment.children", "environment.dogs",
                                        "environment.cats", "distance_y", "hours.monday", "hours.tuesday",
                                        "hours.wednesday", "hours.thursday", "hours.friday",
                                        "hours.saturday", "hours.sunday"])

df_copy["description"] = df_copy["description"].fillna("No description available")
df_copy["mission_statement"] = df_copy["mission_statement"].fillna("No mission statement provided")
df_copy["social_media.facebook"] = df_copy["social_media.facebook"].fillna("No Facebook page")
df_copy["phone"] = df_copy["phone"].fillna("No phone listed")


In [86]:
df_copy.fillna({
    "primary_photo_cropped.full": "No photo available",
    "adoption.url": "Not provided",
    "adoption.policy": "Not specified"
}, inplace=True)

In [87]:
print(df_copy.isnull().sum())

id                               0
Shelter ID                       0
type                             0
species                          0
age                              0
gender                           0
size                             0
breeds.primary                   0
breeds.mixed                     0
breeds.unknown                   0
attributes.spayed_neutered       0
attributes.house_trained         0
attributes.special_needs         0
attributes.shots_current         0
contact.address.city             0
contact.address.state            0
contact.address.postcode         0
status                           0
status_changed_at                0
published_at                     0
distance_x                       0
primary_photo_cropped.full       0
description                      0
name                           257
email                          380
phone                            0
url                            257
website                       1798
mission_statement   

Optimizing Adoption Rates :

🚀Identify which breeds, ages, or sizes get adopted faster.

In [102]:
df_copy["breeds.primary"].value_counts().head(60)


breeds.primary
Mixed Breed                            1252
Pit Bull Terrier                       1089
Labrador Retriever                      576
German Shepherd Dog                     460
Shepherd                                387
Chihuahua                               288
Terrier                                 282
Australian Cattle Dog / Blue Heeler     259
Great Pyrenees                          249
American Staffordshire Terrier          215
Husky                                   209
Black Labrador Retriever                137
Australian Shepherd                     121
Siberian Husky                          116
Border Collie                           115
Boxer                                   115
Catahoula Leopard Dog                   111
Retriever                               106
Black Mouth Cur                         102
Cattle Dog                               88
Hound                                    87
Anatolian Shepherd                       83
American Bulldog 

 Mixed Breed pets are the most commonly adopted, followed by Pit Bull Terriers and Labrador Retrievers.

In [91]:
df_copy["age"].value_counts(normalize=True) *100

age
Adult     54.244461
Young     27.781927
Baby      12.223052
Senior     5.750560
Name: proportion, dtype: float64

1️⃣ Adults make up the majority (54%) → Most adopters seem to prefer mature pets rather than young ones, possibly due to their established personality and training.

2️⃣ Young pets (28%) are in high demand → They are still energetic and adaptable, making them popular among adopters looking for playful companions.

3️⃣ Babies (12%) are adopted but at a lower rate → This could be due to the responsibilities of raising a young pet or fewer available baby animals overall.

4️⃣ Seniors (5%) struggle the most → Older pets might face challenges in adoption due to health concerns or preconceived biases.


In [94]:
df_copy["size"].value_counts()

size
Medium         3838
Large          2893
Small          1172
Extra Large     131
Name: count, dtype: int64

1️⃣ Medium-sized pets dominate (3838 adoptions) 

2️⃣ Large pets (2893 adoptions) are still popular 

3️⃣ Small pets (1172 adoptions) rank lower 

4️⃣ Extra-large pets (131 adoptions) struggle the most 

🎯 Marketing Strategies for Overlooked Pets

1️⃣ Highlight Senior Pets' Unique Qualities

- Showcase their calm temperament: Many senior pets are well-trained and less demanding than young ones.

- Promote their health condition transparently: Ensuring adopters know they're healthy or manageable with proper care.

- Offer adoption incentives: Reduced fees or free vet checkups could encourage more interest.

2️⃣ Spotlight Less Common Breeds

- Tell their story: Personalized bios make less popular breeds stand out.

- Social media campaigns: Targeted ads featuring these pets can improve visibility.

- Breed education posts: Many adopters simply don’t know much about rare breeds, so informing them can increase interest.

3️⃣ Optimize Shelter Listings & Photos

- Prioritize high-quality pictures: Studies show pets with clear, appealing photos get adopted faster.

- Improve descriptions: Avoid generic details; instead, highlight personality traits.

- Use video clips: A short video showing a pet’s temperament can make a huge difference

4️⃣ Community Engagement & Partnerships

- Collaborate with influencers or local businesses: Exposure through partnerships can make overlooked pets more visible.

- Host themed adoption events (e.g., "Senior Pet Week" or "Hidden Gems Adoption Days").

- Work with veterinarians: Trusted recommendations can boost senior pet adoptions.



